In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate

# Koala 7B loading in 8bit on a T4

In [3]:
!nvidia-smi

Wed Apr 19 20:17:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.04              Driver Version: 531.29       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:01:00.0  On |                  N/A |
| 40%   49C    P0              132W / 370W|   1549MiB / 24576MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
import textwrap

/home/uns/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")

base_model = LlamaForCausalLM.from_pretrained(
    "samwit/koala-7b",
    load_in_8bit=True,
    device_map='auto',
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/uns/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary /home/uns/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/uns/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('unix')}
  warn(msg)
/home/uns/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('VSCODE_WSL_EXT_LOCATION/up')}
  warn(msg)
/home/uns/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/uns/go/bin')}
  warn(msg)
/home/uns/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/uns/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py

In [6]:
!nvidia-smi

Wed Apr 19 20:18:13 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.04              Driver Version: 531.29       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:01:00.0  On |                  N/A |
| 58%   50C    P2              132W / 370W|   9462MiB / 24576MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
import textwrap

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=712,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.1
)

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

## Run it as a HF model

In [8]:
import json
import os.path as osp
from typing import Union

class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False, runner: str = 'google-colab'):
        self._verbose = verbose
        
        base_path = '../templates/'
        if runner == 'google-colab':
            base_path = '/content/'
        
        
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join(base_path, f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

question_prompter = Prompter(template_name="generate_question", runner="local")
answer_prompter = Prompter(template_name="generate_answer", runner="local")


dataset = []

In [9]:
import re

def extract_response(text: str):
    splitted = text.split('### Response:\n')
    return splitted[1].strip()



def response_to_array(text: str):
    questions = text.rsplit('\n');
    pattern = r'^[1|2|3]\.\s+'
    qs_cleaned = [];
    
    for q in questions: 
        q_cleaned = re.split(pattern, q)[1]
        qs_cleaned.append(q_cleaned)
    
    return qs_cleaned

In [10]:
context= 'Smart city multi-services kiosque is a terminal which replaced a Parking meter ("horodateur" in French). The difference between a parking meter and a smart city multi-services kiosque is that last one offers the following functions besides the possibility to pay for parking services: maps, timetables, guides, advertisements, tickets for events, coupons which offer discounts in shops etc. Examples of Smart city mult-services kiosques are: StradaPAL and T-PAL module.'
q_prompt = question_prompter.generate_prompt(
    '',
    context
)
output = pipe(q_prompt)
print(wrap_text_preserve_newlines(output[0]['generated_text']))

questions = extract_response(output[0]['generated_text'])

questions_array = response_to_array(questions)

Below is an instruction that describes a task, paired with an input that provides further context. Write a
response that appropriately completes the request.

### Instruction:
 Generate 3 questions for the following passage

### Input:
 Within Flowbird. Smart city multi-services kiosque is a terminal which replaced a Parking meter ("horodateur"
in French). The difference between a parking meter and a smart city multi-services kiosque is that last one
offers the following functions besides the possibility to pay for parking services: maps, timetables, guides,
advertisements, tickets for events, coupons which offer discounts in shops etc. Examples of Smart city mult-
services kiosques are: StradaPAL and T-PAL module.

### Response:
 1.   What are some examples of additional functions offered by smart city multi-services kiosques beyond
parking?
2.   How do smart city multi-services kiosques differ from traditional parking meters?
3.   What are some potential benefits of using smart city 

In [11]:
a_prompt = answer_prompter.generate_prompt(
    context,
    questions
)
output = pipe(a_prompt)
print('\n\n')
print(wrap_text_preserve_newlines(output[0]['generated_text']))

answers = extract_response(output[0]['generated_text'])

answers_array = response_to_array(answers)




Below is an instruction that describes a context. Answer 3 questions from input base on the provided context.

### Instruction:
 Smart city multi-services kiosque is a terminal which replaced a Parking meter ("horodateur" in French). The
difference between a parking meter and a smart city multi-services kiosque is that last one offers the
following functions besides the possibility to pay for parking services: maps, timetables, guides,
advertisements, tickets for events, coupons which offer discounts in shops etc. Examples of Smart city mult-
services kiosques are: StradaPAL and T-PAL module.

### Input:
1.   What are some examples of additional functions offered by smart city multi-services kiosques beyond
parking?
2.   How do smart city multi-services kiosques differ from traditional parking meters?
3.   What are some potential benefits of using smart city multi-services kiosques over traditional parking
meters?

### Response:

1.   Some examples of additional functions offered by

extract question array
extract answer array

make JSON

for 

```
{
    "instruction": "${question}",
    "input": "",
    "output": "${answer}"
}
```



In [12]:
print(questions_array)

print(answers_array)


for i, q in enumerate(questions_array):
    data = {
        "instruction": questions_array[i],
        "input": "In context of Flowbird Group",
        "output": answers_array[i]
    }
    dataset.append(data);

print(dataset)



['What are some examples of additional functions offered by smart city multi-services kiosques beyond parking?', 'How do smart city multi-services kiosques differ from traditional parking meters?', 'What are some potential benefits of using smart city multi-services kiosques over traditional parking meters?']
['Some examples of additional functions offered by smart city multi-services kiosques beyond parking include maps, timetables, guides, advertisements, and tickets for events.', 'Smart city multi-services kiosques differ from traditional parking meters in that they offer additional functions beyond just the ability to pay for parking services.', 'Potential benefits of using smart city multi-services kiosques over traditional parking meters include convenience, accessibility, and improved customer experience.']
[{'instruction': 'What are some examples of additional functions offered by smart city multi-services kiosques beyond parking?', 'input': 'In context of Flowbird Group', 'out

OLD STUF

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: \
What are the difference between Llamas, Alpacas and Koalas?')
print(wrap_text_preserve_newlines(output[0]['generated_text']))


In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: \
Write a short note to Sam Altman giving reasons to open source GPT-4')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: What is the capital of England? \n')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: Write a story about a Koala playing pool and beating all the camelids.')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: Write a story about a Koala playing pool and beating all the camelids.
Once upon a time, in a far-off land where animals could talk, there lived a koala named Koko. Koko was known
for his love of playing pool and his exceptional skills at it. One day, he decided to challenge all the other
animals in the kingdom to a game of pool.

The first animal he challenged was a camel named Cammy. Cammy accepted the challenge and they played a match.
At first, Koko thought he had an easy win as he had been practicing for weeks. But to his surprise, Cammy
proved to be a formidable opponent. The two played several games back and forth, with neither one able to gain
the upper hand.

Next up was a llama named Lola. She too accepted the challenge and they played a match. Once again, Koko
thought he would have an easy win but Lola's quick reflexes and strategic thinking made her a tough opponent.
They played several games back and forth, with both players giving their be

In [ ]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: As an AI do you like the Simpsons? What dow you know about Homer?')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: As an AI do you like the Simpsons? What dow you know about Homer?
As an AI, I don't have personal preferences or feelings. However, I can tell you that The Simpsons is a
popular animated television show created by Matt Groening. It first aired in 1989 and has since become one of
the longest-running TV shows in history. Homer Simpson is one of the main characters on the show, he is a
lazy, accident-prone father who often gets himself into trouble with his family and friends. He is known for
his catchphrase "Doh!" and his love for beer.
CPU times: user 26 s, sys: 74.6 ms, total: 26.1 s
Wall time: 30.5 s
